# Radar Travel Time

Agustín Vallejo Villegas

June 2021

In this notebook we calculate the light distance between pairs of observatories and asteroid 99942 Apophis (one antenna emitting the signal and the other one on the recieving end). Angular size and speed for Apophis is also calculated. All this in order to better grasp where the ideal site of observation would be for this event.

### Libraries

In [1]:
"BASIC LIBRARIES"
import numpy as np
import pylab as plt
import pandas as pd
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')

In [2]:
"ASTROPY"
!pip install astroquery
from astroquery.jplhorizons import Horizons

from astropy.time import Time
from astropy.table import Table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun
from astropy.constants import c
import astropy.units as u

     |████████████████████████████████| 4.4MB 29.3MB/s 
     |████████████████████████████████| 808kB 39.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 3.2MB 37.7MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-cp37-none-any.whl size=3864 sha256=5871d40bf44294772572768856384870dfd84a5dc40dd161a31883e3cd1c6a56
  Stored in directory: /root/.cache/pip/wheels/54/ca/c7/3db47cc5c748286db22a7fab43ccf985903d2b9ca119de16ab
Successfully built mimeparse


In [3]:
"TQDM"
!pip install tqdm
from tqdm import tqdm

## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
!ls "/gdrive/MyDrive/Apophis/Data"

 AllRadiotelescopes2.xlsx  'AllRadiotelescopes .xlsx'
 AllRadiotelescopes3.xlsx


In [ ]:
continents = ['Africa',	'Antarctica',	'Asia',	'Australia',	'Europe',	'North America',	'South America',
              'Arctic Ocean',	'Atlantic Ocean', 'Indian Ocean',	'Pacific Ocean']
radio =  pd.read_excel("/gdrive/MyDrive/Apophis/Data/AllRadiotelescopes3.xlsx",sheet_name='North America',index_col=0)

radio_asia = pd.read_excel("/gdrive/MyDrive/Apophis/Data/AllRadiotelescopes3.xlsx",sheet_name='Asia',index_col=0)

"""
n = 6  # CARMA
n = 10 # Goldstone
n = 11 # Green Bank Interferometer
n = 12 # Green Bank Telescope
n = 12 # Green Bank 140 ft
n = 18 # Leuschner Observatory
n = 20 # Morehead State University 21m
n = 22 # OVRO 40 meter Telescope
n = 28 # VLA
n = 30 # VLBA
n = 31 # PARI
"""

'\nn = 6  # CARMA\nn = 10 # Goldstone\nn = 11 # Green Bank Interferometer\nn = 12 # Green Bank Telescope\nn = 12 # Green Bank 140 ft\nn = 18 # Leuschner Observatory\nn = 20 # Morehead State University 21m\nn = 22 # OVRO 40 meter Telescope\nn = 28 # VLA\nn = 30 # VLBA\nn = 31 # PARI\n'

In [ ]:
radio_asia

,Name,Location,Frequency Range,Remarks,Longitude,Latitude
0,Delingha 13.7 m,"Delingha, Qinghai, China",85–115 GHz,Dish diameter: 13.7 m. Site altitude: 3200 m. ...,NaN,NaN
1,Sheshan,"Shanghai, China",1660 MHz,25 m. Operated by SHAO (Shanghai Astronomical ...,121.187882,31.096323
2,Nanshan 25m,"Ürümqi, China",1.4–18 GHz,L/C/S/X band receivers. 70 km south to Ürümqi....,NaN,NaN
3,Primeval Structure Telescope (PaST),"Xinjiang, China",50–200 MHz,PaST is planned to be an array of some ten-tho...,86.716000,42.924200
4,Chinese Spectral Radio Heliograph (CSRH),"Inner Mongolia, China",0.4–15 GHz,CSRH is an interferometer under construction i...,NaN,NaN
5,Miyun Synthesis Radio Telescope (MSRT),"Miyun, China",232 MHz,Array of 28 9-meter dishes.,NaN,NaN
6,Miyun 50m Radio Telescope,"Miyun, China",2–12 GHz,Built in 2005.,NaN,NaN
7,Kunming 40m Radio Telescope,"Kunming, China",2–12 GHz,Built in 2006.,NaN,NaN
8,Tian Ma 65m Radio Telescope,"Shanghai, China",1–50 GHz,Built in 2012.[12] Operated by SHAO (Shanghai ...,NaN,NaN
9,Giant Metrewave Radio Telescope (GMRT),"Pune, India",50–1420 MHz,Thirty 45 m wire dishes;[13] largest telescope...,74.049742,19.096517


In [ ]:
fast = radio_asia.iloc[17]
fast

Name               Five hundred meter Aperture Spherical Telescop...
Location                                              Guizhou, China
Frequency Range                                          70–3000 MHz
Remarks            500m radio telescope, the world's largest fill...
Longitude                                                    106.857
Latitude                                                     25.6529
Name: 17, dtype: object

In [ ]:
CDSCC = pd.read_excel("/gdrive/MyDrive/Apophis/Data/AllRadiotelescopes3.xlsx",sheet_name='Australia',index_col=0).iloc[2]
CDSCC

Name               Canberra Deep Space Communication Complex (CDSCC)
Location                   Tidbinbilla, Australian Capital Territory
Frequency Range                                                  NaN
Remarks            1x70 m dish, 3x34 m dishes, operated by CSIRO ...
Longitude                                                    148.982
Latitude                                                    -35.4014
Name: 2, dtype: object

## Ephemerids

In [4]:
"ASTRONOMICAL UNIT"
AU = (1*u.au).to(u.m).value
AU2radii = u.AU.to(u.earthRad)

"TIMES"
# The time window shows when the asteroid is at less than 12 earth radii in intervals of 20min
# Closest approach is at 6 earth radii.
t_ini = Time('2029-04-13 19:00:00',format='iso')
t_max = Time('2029-04-13 22:05:00',format='iso') # Closest approach
t_end = Time('2029-04-14 06:00:00',format='iso')

epochs = {'start':t_ini.value, 'stop':t_end.value, 'step':'20m'}

print("(ERFA warning is because the date is very far into the future)")

(ERFA warning is because the date is very far into the future)


/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  ErfaWarning)


In [13]:
"EPHEMERIDS"
apophis = Horizons(id='99942',epochs=epochs) 
ephemeris = apophis.ephemerides()
ephemeris['delta (Rt)'] = ephemeris['delta']*AU2radii
# vectors = apophis.vectors()

# Ecuatorial coordinates of asteroid
ECUcoords = SkyCoord(ephemeris['RA'], ephemeris['DEC']
                     ,ephemeris['delta'], frame='gcrs')

## Apophis Geographical Coordinates

In [6]:
def get_lon(TIME,RA):
  """
  This function outputs the earth longitude at which an object with a certain RA
  at a certain jd will be at the meridian, or directly overhead.
  """
  T = Time(TIME,format='jd')
  G = T.sidereal_time('apparent', 'greenwich').to(u.deg)
  return RA - G.value

maxlons = get_lon(ephemeris['datetime_jd'],ephemeris['RA'])
maxlats = ephemeris['DEC']
deltas = ephemeris['delta (Rt)']

for i in range(len(maxlons)):
  if maxlons[i] < -180: maxlons[i] += 360

ephemeris['Longitude'] = maxlons
ephemeris['Latitude'] = maxlats

for i in range(1,len(maxlons)):
  if maxlons[i] > maxlons[i-1]:
    j = i-1


maxlons[j] = np.nan
maxlons = np.insert(maxlons,j,[-180])
maxlats = np.insert(maxlats,j,[maxlats[j]])
deltas = np.insert(deltas,j,[deltas[j]])

/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utcut1" yielded 34 of "dubious year (Note 3)"
  ErfaWarning)
/usr/local/lib/python3.7/dist-packages/erfa/core.py:155: ErfaWarning: ERFA function "utctai" yielded 34 of "dubious year (Note 3)"
  ErfaWarning)


In [7]:
print("Closest Approach")
ephemeris.to_pandas().loc[np.where(ephemeris['delta'] == min(ephemeris['delta']))][['datetime_str','delta (Rt)','Longitude','Latitude']]

Closest Approach


,datetime_str,delta (Rt),Longitude,Latitude
8,2029-Apr-13 21:40,5.968741,-38.938478,27.2881


## Light Distance

In [ ]:
# Obtains the Astropy Position Vector of given earth coordinates
pos = lambda x,h: EarthLocation(lon=x['Longitude']*u.deg,lat=x['Latitude']*u.deg,height=h*u.earthRad)

# Transforms an Astropy Vector into a 3-tuple
vec = lambda a:  (a.x.value,
                  a.y.value,
                  a.z.value)

# Obtains the difference between two Astropy Vectors
vecdiff = lambda a,b: (a.x.value-b.x.value,
                       a.y.value-b.y.value,
                       a.z.value-b.z.value)

# Obtains the magnitude of a vector
dist = lambda v: np.linalg.norm(v)

# Calculates the angle between two vectors in degrees
angle = lambda v1,v2: np.rad2deg( np.arccos( np.dot(v1,v2) / (dist(v1) * dist(v2)) ) )

# Divides a distance (with astropy units) between light speed c
light_time = lambda d: (d/c).decompose()

def observatory_pair(obs1,obs2):
    pos1 = pos(obs1,0)
    pos2 = pos(obs2,0)

    # Print the observatories information
    print("FIRST OBSERVATORY:")
    print(obs1)
    print("\nSECOND OBSERVATORY:")
    print(obs2)

    # Initializes the pairing dictionary
    pair = {'date':[],
            'angular_size1':[],
            'angular_size2':[],
            'elevation1':[],
            'elevation2':[],
            }

    # Iterate over the whole transit
    for r,row in ephemeris.to_pandas().iterrows():
        posA = pos(row,row['delta (Rt)'])
        vec1 = vecdiff(posA,pos1) # Vector from obs1 to Apophis
        vec2 = vecdiff(posA,pos2) # Vector from obs2 to Apophis
        dist1 = dist(vec1)*u.earthRad # Magnitudes
        dist2 = dist(vec2)*u.earthRad
        distance = (dist1 + dist2) # Length of signal's path
        time = light_time(distance) # Duration of signal's path

        # Apophis angular size
        angle1 = (450*u.m / dist1).decompose() * 206265 *u.arcsec
        angle2 = (450*u.m / dist2).decompose() * 206265 *u.arcsec

        # Apophis elevation
        elev1 = (90-angle( vec(pos1), vec1 )) *u.deg
        elev2 = (90-angle( vec(pos2), vec2 )) *u.deg

        # Flux Density

        pair['date'].append(row['datetime_str'])
        pair['distance'].append( distance )
        pair['light_time'].append( time )
        pair['angular_size1'].append( angle1 )
        pair['angular_size2'].append( angle2 )
        pair['elevation1'].append( elev1 )
        pair['elevation2'].append( elev2 )

    pair_df = pd.DataFrame(pair)
    pair_df = pair_df.style \
        .apply(lambda x: ["background: yellow" if v == max(x) else "" for v in x], axis = 0) \
        .apply(lambda x: ["background: cyan" if v == min(x) else "" for v in x], axis = 0)
    return pair_df

### Observatory Pairings

In [ ]:
"""
n = 6  # CARMA
n = 10 # Goldstone
n = 11 # Green Bank Interferometer
n = 12 # Green Bank Telescope
n = 12 # Green Bank 140 ft
n = 18 # Leuschner Observatory
n = 20 # Morehead State University 21m
n = 22 # OVRO 40 meter Telescope
n = 28 # VLA
n = 30 # VLBA
n = 31 # PARI
"""

obs1 = radio.iloc[10]
obs2 = radio.iloc[28]
observatory_pair(obs1,obs2)

FIRST OBSERVATORY:
Name                                   Goldstone Radio Telescope
Location                           Mojave Desert, California, US
Frequency Range                                              NaN
Remarks            One of the most sensitive radars in the world
Longitude                                                -116.89
Latitude                                                 35.4267
Name: 10, dtype: object

SECOND OBSERVATORY:
Name                          Very Large Array (VLA)
Location                     Socorro, New Mexico, US
Frequency Range                                  NaN
Remarks            Array of 27 dishes. Part of NRAO.
Longitude                                   -107.618
Latitude                                     34.0787
Name: 28, dtype: object


,date,distance,light_time,angular_size1,angular_size2,elevation1,elevation2
0,2029-Apr-13 19:00,27.40595786308913 earthRad,0.5830631664742173 s,1.0612066715506856 arcsec,1.0628298998354526 arcsec,-66.09423148623813 deg,-63.45291921472423 deg
1,2029-Apr-13 19:20,25.255730344670496 earthRad,0.5373169651630892 s,1.1507450782451056 arcsec,1.1541317819226617 arcsec,-61.36957995388181 deg,-57.463568525009464 deg
2,2029-Apr-13 19:40,23.133302069603374 earthRad,0.49216219418747775 s,1.255250452519562 arcsec,1.2611018332559856 arcsec,-55.16346413422653 deg,-50.406216475175995 deg
3,2029-Apr-13 20:00,21.057057315046528 earthRad,0.4479899799917524 s,1.3775994501842235 arcsec,1.3868836618705116 arcsec,-47.58891983123661 deg,-42.238868179982546 deg
4,2029-Apr-13 20:20,19.055476527982428 earthRad,0.4054062455537981 s,1.5204450330811459 arcsec,1.5344481798670702 arcsec,-38.57646411854577 deg,-32.78048234644379 deg
5,2029-Apr-13 20:40,17.17323680899873 earthRad,0.36536149849198274 s,1.6847556450752679 arcsec,1.705013122299368 arcsec,-27.90543758514714 deg,-21.731766020596694 deg
6,2029-Apr-13 21:00,15.48025197688976 earthRad,0.32934315890562055 s,1.8663616976691448 arcsec,1.8941980508130134 arcsec,-15.261973198112273 deg,-8.71886588617376 deg
7,2029-Apr-13 21:20,14.081462573410597 earthRad,0.299583842230848 s,2.049488057105081 arcsec,2.0847022784855005 arcsec,-0.3776113075174976 deg,6.568200158318987 deg
8,2029-Apr-13 21:40,13.116064709754665 earthRad,0.2790449529096767 s,2.199875029628038 arcsec,2.238625495499007 arcsec,16.683430093191674 deg,24.06336291093865 deg
9,2029-Apr-13 22:00,12.723810022501402 earthRad,0.2706997142153463 s,2.270432162873279 arcsec,2.3048097918787627 arcsec,35.17622896171575 deg,42.93671658997948 deg


In [ ]:
goldstone = radio.iloc[10]
green_bank = radio.iloc[12]
observatory_pair(goldstone,green_bank)

FIRST OBSERVATORY:
Name                                   Goldstone Radio Telescope
Location                           Mojave Desert, California, US
Frequency Range                                              NaN
Remarks            One of the most sensitive radars in the world
Longitude                                                -116.89
Latitude                                                 35.4267
Name: 10, dtype: object

SECOND OBSERVATORY:
Name                                      Green Bank Telescope (GBT)
Location                               Green Bank, West Virginia, US
Frequency Range    0.29-2.6 GHz; 2.95-7.8GHz; 8-10 GHz; 12-15.4 G...
Remarks            World's largest 100-metre (330 ft) fully steer...
Longitude                                                   -79.8398
Latitude                                                     38.4331
Name: 12, dtype: object


,date,distance,light_time,angular_size1,angular_size2,elevation1,elevation2
0,2029-Apr-13 19:00,27.190510832170858 earthRad,0.5784795197838798 s,1.0612066715506856 arcsec,1.0798205156815726 arcsec,-66.09423148623813 deg,-43.75867832806577 deg
1,2029-Apr-13 19:20,24.99896197604886 earthRad,0.5318542048827567 s,1.1507450782451056 arcsec,1.1781223530891527 arcsec,-61.36957995388181 deg,-36.9410186488157 deg
2,2029-Apr-13 19:40,22.836714280987508 earthRad,0.48585227369384465 s,1.255250452519562 arcsec,1.294368919822777 arcsec,-55.16346413422653 deg,-29.353359706321356 deg
3,2029-Apr-13 20:00,20.723492535997167 earthRad,0.440893372120267 s,1.3775994501842235 arcsec,1.4324184580887391 arcsec,-47.58891983123661 deg,-20.79111685999918 deg
4,2029-Apr-13 20:20,18.690125283500244 earthRad,0.39763337899145185 s,1.5204450330811459 arcsec,1.59592752777481 arcsec,-38.57646411854577 deg,-10.957183865202495 deg
5,2029-Apr-13 20:40,16.785452832001482 earthRad,0.3571113743888402 s,1.6847556450752679 arcsec,1.7861637975773936 arcsec,-27.90543758514714 deg,0.5485273382450799 deg
6,2029-Apr-13 21:00,15.086451389874165 earthRad,0.32096503111414637 s,1.8663616976691448 arcsec,1.996534869810365 arcsec,-15.261973198112273 deg,14.192264686090397 deg
7,2029-Apr-13 21:20,13.708405671280296 earthRad,0.2916470374047664 s,2.049488057105081 arcsec,2.2024001525085732 arcsec,-0.3776113075174976 deg,30.33071685995764 deg
8,2029-Apr-13 21:40,12.80077165460839 earthRad,0.2723370769062435 s,2.199875029628038 arcsec,2.3527350865130225 arcsec,16.683430093191674 deg,48.800268760004506 deg
9,2029-Apr-13 22:00,12.503799321536647 earthRad,0.26601897520948603 s,2.270432162873279 arcsec,2.3880187127910264 arcsec,35.17622896171575 deg,68.47592283530162 deg


In [ ]:
observatory_pair(fast,CDSCC)

FIRST OBSERVATORY:
Name               Five hundred meter Aperture Spherical Telescop...
Location                                              Guizhou, China
Frequency Range                                          70–3000 MHz
Remarks            500m radio telescope, the world's largest fill...
Longitude                                                    106.857
Latitude                                                     25.6529
Name: 17, dtype: object

SECOND OBSERVATORY:
Name               Canberra Deep Space Communication Complex (CDSCC)
Location                   Tidbinbilla, Australian Capital Territory
Frequency Range                                                  NaN
Remarks            1x70 m dish, 3x34 m dishes, operated by CSIRO ...
Longitude                                                    148.982
Latitude                                                    -35.4014
Name: 2, dtype: object


,date,distance,light_time,angular_size1,angular_size2,elevation1,elevation2
0,2029-Apr-13 19:00,25.19667710800216 earthRad,0.5360606045084315 s,1.1747219385725833 arcsec,1.136194190990931 arcsec,22.665386059723588 deg,-2.333663827957409 deg
1,2029-Apr-13 19:20,23.368460010449144 earthRad,0.4971651915027352 s,1.2709725269602814 arcsec,1.2210444347758704 arcsec,16.891707618148644 deg,-10.380145106906582 deg
2,2029-Apr-13 19:40,21.637427090045637 earthRad,0.4603373768763058 s,1.3776265637288005 arcsec,1.3141717175054073 arcsec,10.389177241648852 deg,-19.06975590893225 deg
3,2029-Apr-13 20:00,20.03120097830144 earthRad,0.42616483353862233 s,1.493551234106073 arcsec,1.414618717922672 arcsec,3.1555965155330483 deg,-28.534687108582347 deg
4,2029-Apr-13 20:20,18.586467253174725 earthRad,0.3954280490521002 s,1.6151515473422844 arcsec,1.5196709187431572 arcsec,-4.764239310073009 deg,-38.91156577486652 deg
5,2029-Apr-13 20:40,17.35000878592333 earthRad,0.36912233141468015 s,1.7348919225764579 arcsec,1.623888598560906 arcsec,-13.194901651270442 deg,-50.29823236237891 deg
6,2029-Apr-13 21:00,16.377062349133162 earthRad,0.348422845810905 s,1.84024901763937 arcsec,1.718361944183567 arcsec,-21.679923023043642 deg,-62.62660562309111 deg
7,2029-Apr-13 21:20,15.724584013814287 earthRad,0.33454133558793164 s,1.9149035530744423 arcsec,1.7911531437046713 arcsec,-29.287287914895302 deg,-75.10684453568479 deg
8,2029-Apr-13 21:40,15.43890060783059 earthRad,0.32846340639698246 s,1.9438925367710722 arcsec,1.8299717498618848 arcsec,-34.57439063846151 deg,-80.86606042507435 deg
9,2029-Apr-13 22:00,15.542299997696091 earthRad,0.3306632337472126 s,1.9209491710610596 arcsec,1.8267596549566598 arcsec,-36.152786895586814 deg,-70.5093076647403 deg


In [ ]:
f = 8.500*1e9 #Hz
lam = 3e8 / f
np.arcsin(1.22 * lam / 500)*206265

17.76305649254411